# find git repo
[here](https://github.com/despresj/ece_colab)

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-2acff931-0c79-8a06-6996-9315f1f2886c)


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"

Mounted at /content/drive


In [17]:
df = pd.read_csv(f"{proj_dir}data_clean/clean.csv")
column_names = df.columns
df = df.to_numpy()

In [18]:
generator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(98, activation="sigmoid")
])

In [19]:
discriminator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid") # not correct output dim
])

In [11]:
gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [12]:
codings_size = 98
batch_size = 32 # number of data points to return
dataset = tf.data.Dataset.from_tensor_slices(df).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [13]:
# Although this runs it still may have bugs
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print(f"Epoch {epoch + 1} of {n_epochs}")
        for _ in range(1000):
            X_batch = np.random.permutation(df)[:100]
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=(100,98))
            generated_data = generator(noise)
            X_fake_and_real = tf.concat([generated_data, X_batch], axis=0)

            y1 = tf.constant([[0.]] * noise.shape[0] + [[1.]] * X_batch.shape[0])
            y1 = np.reshape(y1, (200, 1))

            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)

In [ ]:
train_gan(gan, df, batch_size, codings_size, n_epochs=100)

In [15]:
import os
import re
gans_saved = os.listdir(f"{proj_dir}saved_models/")
model_number = [int(re.sub("gan", "", x)) for x in gans_saved]
last_model = max(model_number)


In [ ]:
generator.save(f"{proj_dir}saved_models/gan{last_model+1}")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ece884_project/saved_models/gan4/assets


# Need to plot this

In [16]:
noise = tf.random.normal(shape=[batch_size, codings_size])
generated_data = generator(noise)
generated_data

<tf.Tensor: shape=(32, 98), dtype=float32, numpy=
array([[0.6497503 , 0.38689566, 0.08327345, ..., 0.45119828, 0.18972148,
        0.1538311 ],
       [0.28231323, 0.6525909 , 0.522682  , ..., 0.2732747 , 0.36492068,
        0.5067434 ],
       [0.540303  , 0.8580951 , 0.39336583, ..., 0.40970555, 0.7097357 ,
        0.11825743],
       ...,
       [0.4682346 , 0.67449045, 0.38262975, ..., 0.5916238 , 0.2704135 ,
        0.61897606],
       [0.35762483, 0.6386495 , 0.50361025, ..., 0.43676016, 0.27636093,
        0.46890917],
       [0.39817578, 0.3024195 , 0.62583774, ..., 0.17560394, 0.3387249 ,
        0.20464092]], dtype=float32)>

In [94]:
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.DataFrame(df, columns=column_names)
generated_array = generated_data[0, :]
generated_df = pd.DataFrame(np.reshape(generated_array, (1, 98)), columns=column_names)

In [95]:
generated_df.loc[:,"hospnum"]

0    0.64975
Name: hospnum, dtype: float32

In [ ]:
fig, ax = plt.subplots(7, 14, figsize=(50,50))
ax = ax.ravel()

for i, column in enumerate(df.columns):
    df[column].hist(ax=ax[i], color="skyblue")
    # generated_df[:,column].hist(ax=ax[i], color="red")
    